### 관찰 및 아이디어
- $S_i$ 는 delta의 개념으로 볼 수 있다.\
어떤 등불이 날아갈 때 이전에 날아갔던 등불보다 그 속력이 빠르다면, 언젠간 반드시 추월하게 되어있다.\
따라서 연속하는 구간에 대해 $S_j < S_i (i < j)$인 개수를 센다면 정답을 구할 수 있을 것이다.\
여기서 Counting Inversion이 생각났다.
- 잘 생각하면, 구간이 증가하는 구간일 때 추월하는 수가 최대가 된다.
  - 가장 긴 strictly increasing 하는 구간에 대해서 Counting Inversion을 세면 된다.
  - 뭔가 구간 Counting Inversion을 구한다고 봐야 할 것 같은데, 여기서 좀 막힌다. $S$ 를 도대체 어떻게 처리해야 할까?

In [ ]:
import io, os, operator
input = io.BytesIO(os.read(0, os.fstat(0).st_size)).readline

class SegmentTree:
  def __init__(self, L, default, f):
    self._def = default
    self._f = f
    self._len = len(L)
    self._size = _size = 1 << (self._len - 1).bit_length()

    self.L = [default] * (2 * _size)
    self.L[_size:_size + self._len] = L
    for i in reversed(range(_size)):
      self.L[i] = f(self.L[i + i], self.L[i + i + 1])

  def __delitem__(self, idx):
    self[idx] = self._def

  def __getitem__(self, idx):
    return self.L[idx + self._size]

  def __setitem__(self, idx, value):
    idx += self._size
    self.L[idx] = value
    idx >>= 1
    while idx:
      self.L[idx] = self._f(self.L[2 * idx], self.L[2 * idx + 1])
      idx >>= 1

  def __len__(self):
    return self._len

  def query(self, s, e):
    s += self._size
    e += self._size

    l = r = self._def
    while s < e: 
      if s & 1:
        l = self._f(l, self.L[s])
        s += 1
      if e & 1:
        e -= 1
        r = self._f(self.L[e], r)
      s >>= 1
      e >>= 1

    return self._f(l, r)

MAX = 100000
def sol() :
  N, S = map(int, input().split())
  L = list(map(int, input().split()))
  Lzip = {v: i for i, v in enumerate(sorted(set(L)))}
  
  seg = SegmentTree([0] * N, 0, operator.add)
  acc = 0
  ans = 0
  for i, v in enumerate(L): 
    v = Lzip[v]
    if i >= S: 
      v2 = Lzip[L[i - S]] # 제거될 등불의 위치
      acc -= seg.query(v2+1, N)
      seg[v2] -= 1
    acc += seg.query(0, v)
    seg[v] += 1
    ans = max(ans, acc)
  
  print(ans)
sol()

### 풀이
- 아이디어는 거의 맞았고, 구현도 맞는 방향으로 잡았지만 `감소시켜야 하는 값`을 제대로 생각해내지 못했다.
  - $k$ 번째 등불을 고려할 때 $k-S$ 번째 등불을 제거한다면, 구간 $(v, N)$ 의 값도 제거해야 한다.

### 시도해봣던 것
- imos로 변환시켜서 그 변화량을 기록하기
  - 본질적으로 감소시켜야 하는 값이 무엇인지 계산할 수 없었다.